In [1]:
### Get all variables ###
import os, json
import pandas as pd
from os import listdir
from os.path import isfile, join

In [2]:
### Show the numbers of PDFs in each category ###
# count_original_pdf = 0
# original_folders = [x[0] for x in os.walk(os.getcwd() + "/MicrodataFiles/")]

# for each in range(1, len(original_folders)):
#     extracted_pdfs = [f for f in listdir(original_folders[each]) if isfile(join(original_folders[each], f))]
#     count_original_pdf = count_original_pdf + len(extracted_pdfs)
#     print(original_folders[each])
#     print(len(extracted_pdfs))
# print(count_original_pdf)

In [3]:
# ## Show the numbers of variables in each category ###
# count_original_variable = 0
# original_folders = [x[0] for x in os.walk(os.getcwd() + "/ExtractFiles/")]

# for each in range(1, len(original_folders)):
#     extracted_pdfs = [f for f in listdir(original_folders[each]) if isfile(join(original_folders[each], f))]
    
#     for item in extracted_pdfs:
#         try:
#             with open(original_folders[each] + '/' + item, 'rb') as f:
#                 read_title = json.load(f)
#         except:
#             print(item)
        
#         count_original_variable = count_original_variable + len(read_title.keys()) - 1
# print(count_original_variable)

In [26]:
### Show the numbers of JSONs (extracted info files) in each category ###
def read_title_info (Topics_File_Folder, Variables_File_Folder):
    
    variables_table = []
    topics_table = []
    extracted_title_folders = [x[0] for x in os.walk(os.getcwd() + Topics_File_Folder)] 
    count_failed_info_file = 0


    for each in range(1, len(extracted_title_folders)):

        extracted_title_json = [f for f in listdir(extracted_title_folders[each]) if isfile(join(extracted_title_folders[each], f))]
        for each_title_json in extracted_title_json:

            topics_list = []
            try:
                with open(extracted_title_folders[each] + '/' + each_title_json, 'rb') as f:
                    read_title = json.load(f)

                topics_list.append(read_title['Title'].replace('\x0c',''))
                topics_list.append(read_title['Date'])
                topics_list.append(extracted_title_folders[each].split('/')[-1])
                topics_list.append(each_title_json[6:-5])
            except UnicodeDecodeError:
                print("Wrong file type!")
            except:
                print("Cannot read %s file!" %each_title_json)


            each_info_json = 'keyInfo' + each_title_json[5:]
            each_extracted_info_folders = extracted_title_folders[each].replace(Topics_File_Folder, Variables_File_Folder)
            try:
                with open(each_extracted_info_folders + '/' + each_info_json, 'rb') as f:
                    read_info = json.load(f)

                topics_list.append(read_info[list(read_info.keys())[0]])
                for key in range(1, len(read_info.keys())):
                    variables_list = []
                    variables_list.append(list(read_info.keys())[key])
                    variables_list.append(read_info[list(read_info.keys())[key]])
                    variables_list.append(read_title['Title'].replace('\x0c',''))
                    variables_list.append(each_info_json[8:-5])
                    
                    variables_table.append(variables_list)

            except UnicodeDecodeError:
                print("Wrong file type!")

            except:
                print("Cannot read %s file!" %each_info_json)
                count_failed_info_file = count_failed_info_file + 1


            variables_table.append(variables_list)
            topics_table.append(topics_list)
    
    return topics_table, variables_table

In [1]:
if __name__ == '__main__':
    
    Topics_File_Folder = "/ExtractVariableTitle_1/"
    Variables_File_Folder = "/ExtractFiles/"
    topics_table, variables_table = read_title_info(Topics_File_Folder, Variables_File_Folder)
    

In [32]:
variables_df = pd.DataFrame.from_dict(variables_table)
# variables_df
variables_df.columns = ['Variable_lable', 'Variable_def', 'Variable_topic', 'File_id']
variables_df.to_csv("CBSMicrodataVariables_NL.csv", index=None)

In [14]:
topics_df = pd.DataFrame.from_dict(topics_table)
topics_df.columns = ['Title', 'Date', 'Category', 'File_id', 'Notes']
topics_df.head()

,Title,Date,Category,File_id,Notes
0,Productiestatistieken 2017,april-2019-19,industrie-en-energie,ps_waterafval,"Dit documentatierapport beschrijft de main, de..."
1,Productiestatistieken 2017,april-2019-19,industrie-en-energie,ps_industrie,"Dit documentatierapport beschrijft de main, de..."
2,Prodcom; verkopen; industriële producten naar ...,maart-2018-3,industrie-en-energie,prodcom,De bestanden zijn beschikbaar over de perioden...
3,Conjunctuurtest industrie,9-maart-2011,industrie-en-energie,conjunctuurtest-industrie-micordata,None
4,Productiestatistieken 2017,april-2019-19,industrie-en-energie,ps_delfstoffenwinning,"Dit documentatierapport beschrijft de main, de..."


In [15]:
### Normalize Date format ###
new_date = []
for each in topics_df['Date']:
    try:
        splited_date = each.split('-')
    
        if splited_date[0].isdigit() :
            if int(splited_date[0]) < 100: 
                if splited_date[2].isdigit() and int(splited_date[2]) > 1000:
                    new_each = each
                    new_date.append(each)
                else:
                    if splited_date[2][0:4].isdigit():
                        new_each = splited_date[0] + '-' + splited_date[1] + '-' + splited_date[2][0:4]
                        new_date.append(each)
                    else:
                        new_each = splited_date[0] + '-' + splited_date[1][:-4] + '-' + splited_date[1][-4:]
                        new_date.append(each)
            else:
                new_each = splited_date[1] + '-' + splited_date[2] + '-' + splited_date[0]
                new_date.append(each)
                    

        else:
            if splited_date[1].isdigit():
                if splited_date[2].isdigit():
                    new_each = splited_date[2] + '-' + splited_date[0] + '-' + splited_date[1]
                    new_date.append(new_each)
                else:
                    new_each = '00' + '-' + splited_date[0] + '-' + splited_date[1]
                    new_date.append(new_each)
            else:
                if splited_date[1][0:4].isdigit():
                    new_each = '00' + '-' + splited_date[0] + '-' + splited_date[1][0:4]
                    new_date.append(new_each)
                else:
                    if splited_date[2].isdigit():
                        new_each = each[1:0]
                        new_date.append(new_each)
                    else:
                        new_date.append(None)

    except:
        new_date.append(None)
        print("****",each)

**** None
**** None
**** None
**** None
**** None
**** None
**** None
**** None
**** None
**** None
**** None


In [17]:
# topics_df['Date'] = new_date
topics_df.to_csv("CBSMicrodataInfoNL_2.csv",index=None)

In [15]:
variables_df = pd.DataFrame.from_dict(variables_table)
variables_df.to_csv("CBSMicrodataVariablesNL.csv")

In [50]:
### Translated by Google Sheet###
# Read translated file #
# topics_df_EN = pd.read_csv('CBSMicrodataInfoNL_GoogleTrans.tsv', sep='\t')[['Title', 'Date', 'Category', 'Notes', 'Title_EN', 'Date_EN', 'Category_EN', 'Notes_EN']]
# variables_df_EN = pd.read_csv('CBSMicrodataVariablesNL_GoogleTrans.tsv', sep='\t')[['Variable_lable', 'Variable_def', 'Variable_topic', 'Variable_def_EN', 'Variable_topic_EN']]

In [51]:
# import numpy as np
# topics_df_EN = topics_df_EN.replace(np.NaN, '')
# topics_df_EN = topics_df_EN.replace('!', '')
# topics_df_EN = topics_df_EN.replace('#VALUE!', '')
# topics_df_EN

# variables_df_EN = variables_df_EN.replace(np.NaN, '')
# variables_df_EN = variables_df_EN.replace('#VALUE!', '')

In [52]:
# topics_df_EN.to_csv("CBSMicrodataInfoEN.csv", index=None)
# variables_df_EN.to_csv("CBSMicrodataVariablesEN.csv", index=None)